In [1]:
#%%appyter init
from appyter import magic
magic.init(lambda _=globals: _())

import xlrd
import csv
import requests
#import Javascript

In [2]:
%%appyter markdown

# {{ StringField(
    name='title',
    label='Title',
    default='My Title',
).render_value }}


# My Title

In [ ]:
%%appyter code_exec

# if small molecule name inputted, will go through its CSV file and sort by target affinity

import pandas as pd

sm_name = {{ StringField(
        name = 'Small molecule name',
        label = 'Small molecule name',
        description = 'The small molecule name',
        default = "Seliciclib",
)}}

kinome_df = pd.read_excel('http://lincs.hms.harvard.edu/wordpress/wp-content/uploads/2013/11/HMS-LINCS_KinomeScan_Datasets_2018-01-18.xlsx')
kinome_df['sm_hms_id'] = kinome_df['sm_hms_id'].str.replace(r'\D', '') #remove HMSL before the ID
hms_id = kinome_df.loc[kinome_df['sm_name'] == sm_name, 'sm_hms_id']

# remove the row number
hms_id_list = []
for hms_id in hms_id:
    hms_id_list.append(hms_id)
    
hms_id_string = str(hms_id_list[0])

old_url = 'http://lincs.hms.harvard.edu/db/datasets/20000/results?small+molecules=HMS_ID&output_type=.csv'
new_url = old_url.replace("HMS_ID", hms_id_string) # fills in HMS_ID with the correct small molecule 

data = pd.read_csv(new_url)
df = pd.DataFrame(data)

target_affinity_1 = df.loc[df['Binding Class'] == 1, 'HUGO Gene Symbol']
target_affinity_2 = df.loc[df['Binding Class'] == 2, 'HUGO Gene Symbol']
target_affinity_3 = df.loc[df['Binding Class'] == 3, 'HUGO Gene Symbol']
target_affinity_10 = df.loc[df['Binding Class'] == 10, 'HUGO Gene Symbol'] 

#js = Javascript('alert("Invalid input")')

kinase1_list = []
kinase2_list = []
kinase3_list = []

for kinase in target_affinity_1:
    kinase1_list.append(str(kinase))

if len(kinase1_list) != 0:
    print('Kinases with Kd < 100 nM: ')
    kinase1_string = ', '.join(kinase1_list)
    print(kinase1_string)

for kinase in target_affinity_2:
    kinase2_list.append(str(kinase)) 

if len(kinase2_list) != 0:
    print('Kinases with 100 nM ≤ Kd < 1µM: ')
    kinase2_string = ', '.join(kinase2_list)
    print(kinase2_string)
    
for kinase in target_affinity_3:
    kinase3_list.append(str(kinase))
    
if len(kinase3_list) != 0:
    print("Kinases with 1µM ≤ Kd < 10 µM: ")
    kinase3_string = ', '.join(kinase3_list)
    print(kinase3_string)

In [ ]:
%%appyter code_exec

# if kinase inputted, go through small molecules  
# check all the rows for each small molecule - if 'HUGO Gene Symbol' includes the kinase then: 
# then go through all of those small molecules and determine which ones have target affinities of 
# 1, 2, and 3

import pandas as pd

kinase_name = {{ StringField(
        name = 'Kinase name',
        label = 'Kinase name',
        description = 'The kinase name',
        default = "CCNA1",
)}}

old_url = 'http://lincs.hms.harvard.edu/db/datasets/20000/results?small+molecules=HMS_ID&output_type=.csv'

df = []

kinome_df = pd.read_excel('http://lincs.hms.harvard.edu/wordpress/wp-content/uploads/2013/11/HMS-LINCS_KinomeScan_Datasets_2018-01-18.xlsx')
kinome_df['sm_hms_id'] = kinome_df['sm_hms_id'].str.replace(r'\D', '') #remove HMSL before the ID


# retrieve the CSV file for the small molecules, 2-183
for x in range(0, 182):  
    hms_id_string = str(kinome_df.iloc[x]['sm_hms_id'])
       
    #print(hms_id_list)
    #print(x)

    new_url = old_url.replace("HMS_ID", hms_id_string) # fills in HMS_ID with the correct small molecule 
    print(new_url)
    data = pd.read_csv(new_url)
    df.append(pd.DataFrame(data))

# lists for each target affinity for the small molecules 
target_aff1 = []
target_aff2 = []
target_aff3 = []
target_aff1_id = []
target_aff2_id = []
target_aff3_id = []

# go through every datafile for every small molecule 
for sm in df:
    if kinase_name in sm.values: # if the protein kinase was studied for that particular small molecule 
        # determine targ aff for the inputted kinase
        target_aff = sm.loc[sm['HUGO Gene Symbol'] == kinase_name, 'Binding Class'] 
        
        # remove the column number 
        targ_aff_list = []
        for targ in target_aff:
            targ_aff_list.append(targ)
        target_aff = targ_aff_list[0]
        
        # add small molecule to the appropriate list 
        if target_aff == 1:
            target_aff1.append(sm.iloc[1]['HMSL Small Mol Name'])
            hms_id = str(sm.iloc[1]['HMSL Small Mol HMS LINCS ID'])   
            head, sep, tail = hms_id.partition('-') # head is the hms_id of the small molecule 
            target_aff1_id.append(head) # HMS ID of the small molecule
        elif target_aff == 2:
            target_aff2.append(sm.iloc[1]['HMSL Small Mol Name'])
        elif target_aff == 3:
            target_aff3.append(sm.iloc[1]['HMSL Small Mol Name'])
        else:
            continue
    
print("Small molecules that bind to the inputted kinase with Kd < 100 nM: ")
print(target_aff1)
print("Small molecules that bind to the inputted kinase with 100 nM ≤ Kd < 1µM: ")
print(target_aff2)
print("Small molecules that bind to the inputted kinase with 1µM ≤ Kd < 10 µM: ")
print(target_aff3)